In [2]:
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer,text_to_word_sequence
import tensorflow as tf
from utils import f1

In [26]:
df = pd.read_csv("train.csv",usecols=["text","target"]);df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [36]:
df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [ ]:
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(df.text)

In [ ]:
word_index = tokenizer.word_index
vocab_size = len(word_index)

In [ ]:
max_length = 300
sequences = tokenizer.texts_to_sequences(df.text)
padded = pad_sequences(sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)

In [55]:
X_train,X_test,y_train,y_test = train_test_split(padded,df.target, random_state = 0)
X_train.shape,X_test.shape

((5709, 300), (1904, 300))

In [66]:
embedding_dim = 100
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),loss='binary_crossentropy',metrics=['acc',f1])
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 300, 100)          2270200   
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 64)                34048     
_________________________________________________________________
dropout_13 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_26 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 129       
Total params: 2,312,697
Trainable params: 2,312,697
Non-trainable params: 0
_________________________________________________________________


In [67]:
num_epochs = 3
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test), verbose=1)

print("Training Complete")

Epoch 1/3
179/179 [==============================] - 9s 50ms/step - loss: 0.5423 - acc: 0.7234 - f1: 0.5361 - val_loss: 0.4400 - val_acc: 0.8036 - val_f1: 0.7156
Epoch 2/3
179/179 [==============================] - 8s 44ms/step - loss: 0.2758 - acc: 0.8891 - f1: 0.8600 - val_loss: 0.4737 - val_acc: 0.7962 - val_f1: 0.7485
Epoch 3/3
179/179 [==============================] - 8s 44ms/step - loss: 0.1221 - acc: 0.9587 - f1: 0.9497 - val_loss: 0.6082 - val_acc: 0.7810 - val_f1: 0.7356
Training Complete


In [63]:
from sklearn.metrics import classification_report
preds = model.predict_classes(X_test)
print(classification_report(y_test,preds))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
              precision    recall  f1-score   support

           0       0.80      0.84      0.82      1107
           1       0.76      0.71      0.74       797

    accuracy                           0.79      1904
   macro avg       0.78      0.78      0.78      1904
weighted avg       0.79      0.79      0.79      1904

